In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

# OpenCV: BGR; Matplot: RGB
def show_image(img):
    RGB_im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(RGB_im)
    plt.show()

In [ ]:
def extract_img(img):
    new_img = img[:, :, 1] # The G part is clearer
    #show_image(new_img)
    return new_img

## Save the Image
Called by split_img()

In [ ]:
def save_img(img, filename):
    name = filename + ".jpg"
    cv2.imwrite(name, img)

In [ ]:
def split_img(img, path):
    if img is None:
        return

    rows = img.shape[1]

    step = rows//4
    # if step is too small which means the picture is too small, skip it
    if (step < 100):
        return
    
    name = os.path.splitext(path)
    
    old_row = 0
    for i in range(1, 5):
        small_img = img[:, old_row:step*i]
        #show_image(small_img)
        old_row = step*i
        save_img(small_img, name[0] + "-" + str(i))

In [ ]:
DIR = 'test_images/'

for root, subdirs, files in os.walk(DIR):
    print("root:", root)
    for filename in files:
        file_path = os.path.join(root, filename)
        if file_path.endswith(".jpg"):
            print(file_path)
            img = cv2.imread(file_path, cv2.IMREAD_COLOR)
            #show_image(img)
            G_img = extract_img(img)
            split_img(G_img, file_path)

print("Done")